# IBM Applied Data Science Capstone Project


## Import all the necessry libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes

# convert an address into latitude and longitude values
#from geopy.geocoders import Nominatim 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#Install and import folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

## Use Foursquare Client ID and Secret

In [2]:
# your Foursquare ID
CLIENT_ID = 'YURAW5R5XNEOB3HBPIL1TKN4MB2HM2VBXGPFEZATUZ2K23K1'
# your Foursquare Secret
CLIENT_SECRET = 'BG0ZCMUVHPLYEZZCOF4A3YBG0HERLP5UYLOJDJNUZRZA2SBZ'
VERSION = '20202204'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YURAW5R5XNEOB3HBPIL1TKN4MB2HM2VBXGPFEZATUZ2K23K1
CLIENT_SECRET:BG0ZCMUVHPLYEZZCOF4A3YBG0HERLP5UYLOJDJNUZRZA2SBZ


## Install geopy and import Nominatim to convert address into latitude & longitude

In [3]:
!conda install -c conda-forge geopy --yes

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.21.0-py_0 conda-forge


geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.21.0         | 58 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


## Give the city name and use Nominatim to find the latitude & longitude

In [127]:
# define the city and get its latitude & longitude 

address = 'Mumbai, India'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate : {}, {}.'.format(latitude, longitude))

The geograpical coordinate : 18.9387711, 72.8353355.


In [87]:
neighborhood_latitude = 28.6141793
neighborhood_longitude = 77.2022662

In [139]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 100000 # define radius
#search_query = 'Hotel'
# create URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    #search_query,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/search?&client_id=YURAW5R5XNEOB3HBPIL1TKN4MB2HM2VBXGPFEZATUZ2K23K1&client_secret=BG0ZCMUVHPLYEZZCOF4A3YBG0HERLP5UYLOJDJNUZRZA2SBZ&v=20202204&ll=18.9387711,72.8353355&radius=100000&limit=1000'

In [140]:
# results display is hidden for report simplification 
results = requests.get(url).json()
print ("Results\n", results)

Results
 {'meta': {'code': 200, 'requestId': '5eadbb607828ae001c44fb58'}, 'response': {'venues': [{'id': '4c722cd4376da093b9c1a5c6', 'name': 'Shivala', 'location': {'address': 'Walchand Hirachand Marg,', 'crossStreet': 'Fort', 'lat': 18.938517451584296, 'lng': 72.83551366851701, 'labeledLatLngs': [{'label': 'display', 'lat': 18.938517451584296, 'lng': 72.83551366851701}], 'distance': 33, 'postalCode': '400001', 'cc': 'IN', 'city': 'Mumbai', 'state': 'Mahārāshtra', 'country': 'India', 'formattedAddress': ['Walchand Hirachand Marg, (Fort)', 'Mumbai 400001', 'Mahārāshtra', 'India']}, 'categories': [{'id': '4bf58dd8d48988d1d3941735', 'name': 'Vegetarian / Vegan Restaurant', 'pluralName': 'Vegetarian / Vegan Restaurants', 'shortName': 'Vegetarian / Vegan', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1588444073', 'hasPerk': False}, {'id': '4babe4fff964a520f8d23ae3', 'name': 'Chhatrapati Shivaji Maharaj T

In [141]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Venues into Dataframe

In [142]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,4c722cd4376da093b9c1a5c6,"Walchand Hirachand Marg,",IN,Mumbai,India,Fort,33,"[Walchand Hirachand Marg, (Fort), Mumbai 40000...","[{'label': 'display', 'lat': 18.93851745158429...",18.938517,72.835514,400001,Mahārāshtra,Shivala,v-1588444073,NaN
1,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4babe4fff964a520f8d23ae3,Dr. Dadabhai Naoroji Road,IN,Mumbai,India,D.N.Road,146,"[Dr. Dadabhai Naoroji Road (D.N.Road), Mumbai ...",NaN,18.940088,72.835257,400001,Mahārāshtra,Chhatrapati Shivaji Maharaj Terminus,v-1588444073,32965357
2,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4e33a472227111ae76912f24,Opp gpo,IN,Mumbai,India,NaN,72,"[Opp gpo, Mumbai, Mahārāshtra, India]","[{'label': 'display', 'lat': 18.93821427897314...",18.938214,72.835697,NaN,Mahārāshtra,Pancham Puriwala,v-1588444073,NaN
3,"[{'id': '4bf58dd8d48988d172941735', 'name': 'P...",False,4b0587d1f964a520eca222e3,Saint George Road,IN,Mumbai,India,off Walchand Hirachand Road,190,[Saint George Road (off Walchand Hirachand Roa...,"[{'label': 'display', 'lat': 18.93859777347305...",18.938598,72.837138,400 001,Mahārāshtra,General Post Office (GPO),v-1588444073,NaN
4,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4b7b66b2f964a52011622fe3,D N Rd.,IN,Mumbai,India,Opposite Mumbai CST railway station,90,[D N Rd. (Opposite Mumbai CST railway station)...,"[{'label': 'display', 'lat': 18.93898459503898...",18.938985,72.834504,400001,Mahārāshtra,McDonald's,v-1588444073,NaN


In [143]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Shivala,Vegetarian / Vegan Restaurant,"Walchand Hirachand Marg,",IN,Mumbai,India,Fort,33,"[Walchand Hirachand Marg, (Fort), Mumbai 40000...","[{'label': 'display', 'lat': 18.93851745158429...",18.938517,72.835514,400001,Mahārāshtra,4c722cd4376da093b9c1a5c6
1,Chhatrapati Shivaji Maharaj Terminus,Train Station,Dr. Dadabhai Naoroji Road,IN,Mumbai,India,D.N.Road,146,"[Dr. Dadabhai Naoroji Road (D.N.Road), Mumbai ...",NaN,18.940088,72.835257,400001,Mahārāshtra,4babe4fff964a520f8d23ae3
2,Pancham Puriwala,Indian Restaurant,Opp gpo,IN,Mumbai,India,NaN,72,"[Opp gpo, Mumbai, Mahārāshtra, India]","[{'label': 'display', 'lat': 18.93821427897314...",18.938214,72.835697,NaN,Mahārāshtra,4e33a472227111ae76912f24
3,General Post Office (GPO),Post Office,Saint George Road,IN,Mumbai,India,off Walchand Hirachand Road,190,[Saint George Road (off Walchand Hirachand Roa...,"[{'label': 'display', 'lat': 18.93859777347305...",18.938598,72.837138,400 001,Mahārāshtra,4b0587d1f964a520eca222e3
4,McDonald's,Fast Food Restaurant,D N Rd.,IN,Mumbai,India,Opposite Mumbai CST railway station,90,[D N Rd. (Opposite Mumbai CST railway station)...,"[{'label': 'display', 'lat': 18.93898459503898...",18.938985,72.834504,400001,Mahārāshtra,4b7b66b2f964a52011622fe3


## Delete the unnecessary columns

In [144]:

# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','id'], axis=1)
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3


,name,categories,address,lat,lng,postalCode,state
0,Shivala,Vegetarian / Vegan Restaurant,"Walchand Hirachand Marg,",18.938517,72.835514,400001,Mahārāshtra
1,Chhatrapati Shivaji Maharaj Terminus,Train Station,Dr. Dadabhai Naoroji Road,18.940088,72.835257,400001,Mahārāshtra
3,General Post Office (GPO),Post Office,Saint George Road,18.938598,72.837138,400 001,Mahārāshtra
4,McDonald's,Fast Food Restaurant,D N Rd.,18.938985,72.834504,400001,Mahārāshtra
6,CST Bus Stand,Bus Station,"Next to General Post Office, Off. CST Station",18.939009,72.835125,400001,Mahārāshtra
9,India Government Mint - SPMCIL,Factory,Mint Road,18.939067,72.834899,400 001,Mahārāshtra
10,Town House Cafe,Bar,"# 16, Near Sterling Cinema, Murzban Road, Azad...",18.938550,72.833464,400001,Mahārāshtra
12,Aaram Vada Pav,Snack Place,Dadabhai Naoroji Road,18.939616,72.834935,400001,Mahārāshtra
17,B.M.C. Headquarters,City Hall,Mahapalika Marg,18.940565,72.834098,400001,Mahārāshtra
18,Hbo - Hey Belly Ola,Fast Food Restaurant,138 Empire building,18.938610,72.834540,400001,Mahārāshtra


## Remove rows which are not necessary 

In [152]:

# delete rows which its category is not office
array= ['Office']
dataframe4= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
dataframe4

,name,categories,address,lat,lng,postalCode,state
91,Hembina Enterprises / The Cotton Company,Office,66 Bazargate Street,18.937330,72.835777,400001,Mahārāshtra
100,Jai News Agency Office,Office,dn road fort,18.939778,72.834469,400001,Mahārāshtra
111,Kulkarni Patil Bhavan,Office,"14, Murzban Road,",18.938815,72.833751,400001,Mahārāshtra
120,HPCL,Office,Hindustan Bhavan,18.938442,72.835390,400001,Mahārāshtra


## Generate map to visualize 

In [151]:
# Generate map to visualize offices in the neighborhood
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(dataframe4['lat'], dataframe4['lng'], 
                                           dataframe4['name'], dataframe4['categories'],\
                                               dataframe4['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map